In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests

from sklearn.metrics import mean_squared_error

In [3]:
test1_df = pd.read_csv('SE_rents2018_test1.csv', index_col=0)
test2_df = pd.read_csv('SE_rents2018_test2.csv', index_col=0)
test3_df = pd.read_csv('SE_rents2018_test3.csv', index_col=0)
train_df = pd.read_csv('SE_rents2018_train.csv', index_col=0)
# bedrooms, bathrooms, size_sqft, and year_built use for linear model.
# borough, neighborhood, has_doorman, and size_sqft for non-linear model. 

In [4]:
url = 'https://api.census.gov/data/2017/acs/acs5?'
params = {'get' : 'NAME,B19013_001E,B25058_001E',
          'for' : 'zip code tabulation area:*',}

r = requests.get(url, params=params)
print(r.url)

https://api.census.gov/data/2017/acs/acs5?get=NAME%2CB19013_001E%2CB25058_001E&for=zip+code+tabulation+area%3A%2A


In [5]:
income_df = pd.DataFrame(r.json()[1:], columns=r.json()[0])
income_df.head(10)

,NAME,B19013_001E,B25058_001E,zip code tabulation area
0,ZCTA5 00601,11757,291,00601
1,ZCTA5 00602,16190,304,00602
2,ZCTA5 00603,16645,306,00603
3,ZCTA5 00606,13387,185,00606
4,ZCTA5 00610,18741,322,00610
5,ZCTA5 00612,17744,310,00612
6,ZCTA5 00616,14918,327,00616
7,ZCTA5 00617,17157,375,00617
8,ZCTA5 00622,16727,392,00622
9,ZCTA5 00623,16401,352,00623


In [6]:

income_df = (income_df
             .rename(
                 columns={
                          'B19013_001E' : 'Median Household Income',
                          'B25058_001E' : 'Median Rent',
                          'zip code tabulation area' : 'addr_zip',
                         }
             )
            )
income_df.head(3)
#income_df.loc['10956']
#income_df.dtypes

,NAME,Median Household Income,Median Rent,addr_zip
0,ZCTA5 00601,11757,291,00601
1,ZCTA5 00602,16190,304,00602
2,ZCTA5 00603,16645,306,00603


In [7]:
income_df['addr_zip'] = income_df['addr_zip'].astype(int)
income_df['Median Rent'] = income_df['Median Rent'].astype(int)
income_df['Median Household Income'] = income_df['Median Household Income'].astype(int)

In [8]:
test1_df.head(5)

,addr_unit,building_id,bedrooms,bathrooms,size_sqft,created_at,addr_street,addr_city,addr_zip,addr_lat,...,has_garden,has_childrens_playroom,rent,no_fee,description,neighborhood,borough,unit,floornumber,line
rental_id,,,,,,,,,,,,,,,,,,,,,
7499437,#1B,18724,1,1.0,550.0,2018-08-22 10:00:37,167 MOTT STREET,New York,10013,40.719799,...,0,0,2700,1,"By Owner, No Fees!\n\nNewly renovated building...",Little Italy,Manhattan,1B,1.0,B
7477903,#C4,403900,1,1.0,750.0,2018-08-15 11:41:23,107 LINDEN STREET,Brooklyn,11221,40.693118,...,0,0,1650,1,this is a beautiful spacious renovated full s...,Bushwick,Brooklyn,C4,4.0,C
7481023,#GARDEN,18175264,2,1.0,800.0,2018-08-15 22:20:14,47 CARROLL STREET,Brooklyn,11231,40.683399,...,1,0,2900,0,"Garden level, pet-friendly, 2 bedroom, 1 bath ...",Columbia St Waterfront District,Brooklyn,GARDEN,NaN,NaN
7479547,#2C,104863,2,2.0,1300.0,2018-08-15 16:05:22,952 COLUMBUS AVENUE,New York,10025,40.799900,...,0,0,4750,1,Ivy Park is a boutique condominium with a uniq...,Upper West Side,Manhattan,2C,2.0,C
7518814,#9,95125,2,1.0,750.0,2018-08-29 12:00:12,424 EAST 116 STREET,New York,10029,40.795350,...,0,0,1800,0,Located across from the New East River Plaza w...,East Harlem,Manhattan,9,NaN,NaN


In [9]:
# # Here is the link to the data set. https://catalog.data.gov/dataset/demographic-statistics-by-zip-code-acfc9
# demographic_by_zipcode_df = pd.read_csv('Demographic_Statistics_By_Zip_Code.csv', index_col=0)
# print(demographic_by_zipcode_df.columns)
# test_merge_df = pd.merge(test_merge_df,demographic_by_zipcode_df, left_on = test_merge_df.addr_zip , right_index=True, how = 'left')


In [10]:
def get_heatmap(df):
    corrmat = df.corr()
    top_corr = corrmat.index
    plt.figure(figsize=(len(df.columns),len(df.columns)))
    df_g = sns.heatmap(df[top_corr].corr(), annot=True, cmap="RdYlGn")
    return df_g

In [ ]:
import numpy

#using a heatmap to select meaningful features
#columns = test1_df[:,0:29]
#target = test1_df.rent
# get_heatmap(test1_df)


# train_merge_df = train_df.merge(right=income_df,how='left', on='addr_zip')

# columns_needed = income_df.columns
# columns_needed = numpy.append (columns_needed, ['rent'])

# print(columns_needed)
# test_heatmap_df = train_merge_df.filter(columns_needed, axis=1)

get_heatmap(train_df)

In [ ]:
# sns.pairplot(test_heatmap_df, size=2.5)
# plt.tight_layout()
# plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


def train_linear_reg_model(X_train, y_train, X_test, y_test):
    """Trains and test linear regression model and returns mean_squared_error of
    y_test and y_predicted. 
    ----------
    X_train : data frame, shape = [n_samples, n_features]
        n_samples the number of samples
        n_features the number of features for each sample.
        
    y_train : data frame, shape = [n_samples]
        n_sample target values.
        
    X_test : data frame, shape = [n_samples, n_features]
        n_samples the number of samples
        n_features the number of features for each sample.
        
    y_test : data frame, shape = [n_samples]
        n_sample target values.
        
    Return 
        (data frame) y_prediction: predictions made from X_test
        (float) mean_squared_error of (y_test and y_prediction)
        (LinearRegression) returns the linear regression model.
    """

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    y_prediction = lr.predict(X_test)
    
    return lr ,y_prediction, mean_squared_error(y_test, y_prediction)

In [ ]:
train_merge_df = train_df.merge(right=income_df,how='left', on='addr_zip')
test1_merge_df = test1_df.merge(right=income_df,how='left', on='addr_zip')
test2_merge_df = test2_df.merge(right=income_df,how='left', on='addr_zip')
test3_merge_df = test3_df.merge(right=income_df,how='left', on='addr_zip')

relfeatures_df = train_df[['size_sqft', 'bathrooms', 'bedrooms', 'has_gym', 'bin', 'addr_zip', 'bbl', 'rent']].copy()

relfeatures_df = train_merge_df[['size_sqft', 'bathrooms', 'bedrooms', 'has_gym', 'bin', 'addr_zip', 'bbl', 'rent']].copy()

mv_percentages = train_merge_df.isna().sum()/len(train_merge_df)*100
mv_percentages

for name, percent in mv_percentages.iteritems():
    if(percent<5 and percent > 0):
        train_merge_df = train_merge_df[pd.notnull(train_merge_df[name])]
train_merge_df.shape

from pandas.api.types import is_numeric_dtype
mv_percentages_step2 = train_merge_df.isna().sum()/len(train_merge_df)*100
for name, percent in mv_percentages_step2.iteritems():
    if(percent>=5):
        print(name + ": " + str(train_merge_df[name].value_counts().idxmax()))
        if(is_numeric_dtype(train_merge_df[name])):
            train_merge_df[name].fillna(train_merge_df[name].mean(), inplace=True)
        else:
            train_merge_df[name].fillna(train_merge_df[name].value_counts().idxmax(), inplace=True)
#removing outliers increased mse


print(relfeatures_df[relfeatures_df["size_sqft"]>5000])
train_df = train_df[train_df["size_sqft"]<5000]
print(relfeatures_df[relfeatures_df["size_sqft"]<5])
train_df = train_df[train_df["size_sqft"]>5]
print(relfeatures_df[relfeatures_df["bathrooms"]>7])
train_df = train_df[train_df["bathrooms"]<7]
print(train_df[train_df["rent"]>30000])

relfeatures_df.shape

In [ ]:
columns_needed = ['bathrooms', 'size_sqft','bbl','floor_count','Median Household Income','rent']
features = ['bathrooms', 'size_sqft','bbl','floor_count','Median Household Income']

train_merge_df = train_merge_df[columns_needed]

# Dataset contains one null value in bin. So I decided to drop it.
train_merge_df = train_merge_df.dropna(axis=0)

# Getting the target columns
train_merge_target = train_merge_df[['rent']]
# Getting the features for training.
train_merge_df = train_merge_df[features]

# Getting the target columns to get accuracy of predictions. 
test1_merge_target = test1_merge_df[['rent']]
# Extracting the columns needed for testing. Useing test1.csv file.
test1_merge_df = test1_merge_df[features]

test1_merge_df = test1_merge_df.fillna(test1_merge_df.median())


lr, y_prediction, mse = train_linear_reg_model(train_merge_df, train_merge_target, test1_merge_df, test1_merge_target)
print("Mean square error of test1: " + str(mse))

In [18]:
# test2_lr_df = test2_df[features]
# test2_rent_predictions = lr.predict(test2_lr_df)
# test2_rent_predictions = pd.DataFrame(test2_rent_predictions, index=test2_lr_df.index, columns=['rent'])
# test2_rent_predictions
# # send to a csv file
# test2_rent_predictions.to_csv('rent_predictions.csv')


KeyError: "['Median Household Income'] not in index"

In [16]:
# Polynomial Regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

poly = PolynomialFeatures()
poly = poly.fit_transform(train_merge_df)
model = Pipeline([('poly', PolynomialFeatures(degree=4)), 
                  ('linear', LinearRegression(normalize=True))])

model = model.fit(train_merge_df, train_merge_target)
test1_predict = model.predict(test1_merge_df)
train_predict = model.predict(train_merge_df)
print("Mean squared error: " + str(mean_squared_error(test1_merge_target, test1_predict)))

print("Test R2 value is: " + str(r2_score(test1_merge_target, test1_predict)))
print("Train R2 value is: " + str(r2_score(train_merge_target, train_predict)))

Mean squared error: 2276794.3982981904
Test R2 value is: 0.69468363638138
Train R2 value is: 0.849387157467093


In [17]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.datasets import make_regression


# for i in range(1, 20):
#     regr = RandomForestRegressor(max_depth=i, random_state=0,
#                                  n_estimators=100)
#     regr.fit(train_merge_df, train_merge_target)

#     test1_regr_predict = regr.predict(test1_merge_df)
#     print(mean_squared_error(test1_merge_target, test1_regr_predict))
    
#     print("R2 value is: " + str(r2_score(test1_merge_target, test1_regr_predict)))
#     train_regr_predict = regr.predict(train_merge_df)
#     print("Train R2 value is: " + str(r2_score(train_merge_target, train_regr_predict)))